In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import LSTM, Dense,Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MultiHeadAttention, Dropout
import tushare as ts
import os

stocks = pd.read_csv('result_final.csv',header=None) 
ts.set_token('b0030b7297c6b297f1db549c7f8c8a080b5f28e2267f07c4741e69ad')
pro = ts.pro_api()
for stock in stocks.values:
    try:
        if stock[0] < 0.7:
            continue
        # data = pro.daily(ts_code=stock[0], start_date='20220101', end_date='20230703')
        data = ts.pro_bar(ts_code=stock[1],freq='1min', start_date='2023-01-01 09:00:00', end_date='2023-06-01 15:00:00')
        data = data[['close', 'vol']]
        data.sort_index(ascending=False, inplace=True)
        data = data.reset_index(drop=True)
        # 计算差值
        data_diff = data.diff().dropna()
        # 数据缩放
        scaler = StandardScaler()
        data_scaled = scaler.fit_transform(data_diff)

        lookback = 50

        # 定义生成器函数
        def generator(data, lookback=10, delay=1, batch_size=20):
            max_index = len(data) - delay - 1
            i = lookback
            while 1:
                if i + delay >= max_index:
                    i = lookback
                samples = np.zeros((batch_size, lookback, data.shape[-1]))
                targets = np.zeros((batch_size, data.shape[-1]))  # targets now is 2D, predict price and volume
                for j in range(batch_size):
                    if i + delay >= max_index:
                        i = lookback
                    rows = np.arange(i - lookback, i)
                    samples[j] = data[rows]
                    targets[j] = data[i + delay]  # target is the next value sequence
                    i += 1
                yield samples, targets

        
        ##lstm
        inputs = Input(shape=(lookback, data.shape[-1]))
        x = LSTM(128, return_sequences=True)(inputs)  # first LSTM layer
        x = Dropout(0.5)(x)
        x = LSTM(128, return_sequences=True)(inputs)  # first LSTM layer
        x = Dropout(0.5)(x)
        x = LSTM(128, return_sequences=False)(x)  # second LSTM layer
        outputs = Dense(data.shape[-1])(x)   # only predict the next value
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer=Adam(), loss='mse')

        train_data = data_scaled[:int(len(data_scaled)*0.9)]
        val_data = list(data_scaled[int(len(data_scaled)*0.1):])

    
         #预测涨跌
        correct_predictions = 0
        total_predictions = 0
        train_gen = generator(train_data, lookback=lookback)
        model.fit(train_gen, epochs=20, steps_per_epoch=20, verbose=0)
        for epoch in range(10):  # rolling train for 30 epochs
            train_gen = generator(train_data, lookback=lookback)
            model.fit(train_gen, epochs=1, steps_per_epoch=20, verbose=0)  # train the model for one epoch

            # move the first data point from val_data to train_data
            train_data = np.concatenate([train_data, np.reshape(val_data.pop(0), (1, -1))])
            # 预测下一个点
            last_sequence = np.reshape(train_data[-lookback:], (1, lookback, data_diff.shape[-1]))
            next_prediction = model.predict(last_sequence, verbose=0)[0, 0]

            # 计算预测值与真实值之间的差距
            if len(val_data) > 0:
                next_real = val_data[0][0]  # the next real value
                error = np.abs((next_prediction - next_real) / next_real)  # the relative error
                print(f'Epoch {epoch + 1}, next_prediction:{next_prediction:.4f},next_real:{next_real:.4f},validation error: {error:.4f}')

                # 对预测结果进行涨跌预测
                prediction_increase = next_prediction > 0
                real_increase = next_real > 0
                if prediction_increase == real_increase:
                    correct_predictions += 1
                total_predictions += 1

        # 计算预测准确率
        accuracy = correct_predictions / total_predictions
        print(f'Prediction accuracy: {accuracy:.4f}')
        new_result = pd.DataFrame([[accuracy,stock[1],stock[2]]], columns=result.columns)
        print(new_result)
        result.append(new_result)
        new_result.to_csv('result.csv', mode='a', header=False, index=False)
    except Exception as e:
        print(e)
        continue 
    
    

